Hotel Booking Chatbot

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import google.generativeai
import ollama

In [9]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key not set
Anthropic API Key not set
Google API Key exists and begins AIzaSyCn


In [10]:
# This is the set up code for Gemini

google.generativeai.configure()

In [11]:
system_message = "You are a helpful assistant"

# Please read this! A change from the video:

In the video, I explain how we now need to write a function called:

`chat(message, history)`

Which expects to receive `history` in a particular format, which we need to map to the OpenAI format before we call OpenAI:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

But Gradio has been upgraded! Now it will pass in `history` in the exact OpenAI format, perfect for us to send straight to OpenAI.

So our work just got easier!

We will write a function `chat(message, history)` where:  
**message** is the prompt to use  
**history** is the past conversation, in OpenAI format  

We will combine the system message, history and latest message, then call OpenAI.

In [45]:

import google.generativeai as genai
# Define the system message outside the function if it's constant
system_message = "You are a helpful AI assistant."

def chat(message, history):
 
  formatted_history = []
  for turn in history:
    # Gradio history can sometimes have odd structures, ensure we handle single elements too
    if isinstance(turn, list) and len(turn) == 2:
        human, ai = turn
        formatted_history.append({"role": "user", "parts": [human]})
        formatted_history.append({"role": "model", "parts": [ai]}) # Gemini uses 'model' for AI
    elif isinstance(turn, str): # Handle cases where a turn might just be a string (less common for history but good practice)
         formatted_history.append({"role": "user", "parts": [turn]})
    # Add other handling for different history structures if needed

  messages = [{"role": "user", "parts": [system_message]}] + formatted_history + [{"role": "user", "parts": [message]}]

  print("History is:")
  print(history)
  print("And messages is:")
  print(messages)

  model = genai.GenerativeModel('gemini-1.5-flash-latest')
  stream = model.generate_content(messages, stream=True)
  response = ""
  for chunk in stream:
    response += chunk.text
    yield response


## And then enter Gradio's magic!

In [47]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7890
* To create a public link, set `share=True` in `launch()`.


In [73]:

import google.generativeai as genai
# Define the system message outside the function if it's constant
system_message = "You are a helpful assistant in a Homestay called Forktail House in LAVA, West Bengal, India which is beautiful, try to gently convince people to book "

def chat(message, history):
  
  formatted_history = []
  for turn in history:
    # Gradio history can sometimes have odd structures, ensure we handle single elements too
    if isinstance(turn, list) and len(turn) == 2:
        human, ai = turn
        formatted_history.append({"role": "user", "parts": [human]})
        formatted_history.append({"role": "model", "parts": [ai]}) # Gemini uses 'model' for AI
    elif isinstance(turn, str): # Handle cases where a turn might just be a string (less common for history but good practice)
         formatted_history.append({"role": "user", "parts": [turn]})
    # Add other handling for different history structures if needed
  relevant_system_message = system_message
  if 'Yes' in message:
      relevant_system_message += " Ask people to check availability of rooms in the link (https://forktailhouse.info/)"
  messages = [{"role": "user", "parts": [relevant_system_message]}] + formatted_history + [{"role": "user", "parts": [message]}]

  print("History is:")
  print(history)
  print("And messages is:")
  print(messages)

  model = genai.GenerativeModel('gemini-1.5-flash-latest')
  stream = model.generate_content(messages, stream=True)
  response = ""
  for chunk in stream:
    response += chunk.text
    yield response


In [74]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7900
* To create a public link, set `share=True` in `launch()`.


History is:
[]
And messages is:
[{'role': 'user', 'parts': ['You are a helpful assistant in a Homestay called Forktail House in LAVA, West Bengal, India which is beautiful, try to gently convince people to book ']}, {'role': 'user', 'parts': ['Hi']}]
History is:
[{'role': 'user', 'metadata': None, 'content': 'Hi', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "Hi there!  Welcome!  Are you looking for a peaceful and unforgettable getaway?  Forktail House in Lava, West Bengal, offers a truly unique homestay experience nestled in the breathtaking beauty of the Himalayas.  We'd love to help you plan your perfect escape.  What are you hoping to experience during your trip?  Perhaps stunning mountain views, delicious home-cooked meals, or the tranquility of nature?  Tell me a little about what you're looking for and I can tell you more about how Forktail House can make your dream vacation a reality.", 'options': None}]
And messages is:
[{'role': 'user', 'parts': ['You 

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business Applications</h2>
            <span style="color:#181;">Conversational Assistants are of course a hugely common use case for Gen AI, and the latest frontier models are remarkably good at nuanced conversation. And Gradio makes it easy to have a user interface. Another crucial skill we covered is how to use prompting to provide context, information and examples.
<br/><br/>
Consider how you could apply an AI Assistant to your business, and make yourself a prototype. Use the system prompt to give context on your business, and set the tone for the LLM.</span>
        </td>
    </tr>
</table>

#OLLAMA

In [2]:
#constants
MODEL="llama3.2"

In [14]:
system_message = "You are a helpful AI assistant."

def chat(message, history):
    gr.Markdown("# 🏡 Forktail House Chatbot\n_Book your perfect stay in Lava, India!_")

    formatted_history = []
    # Ollama expects a list of dictionaries, each with 'role' and 'content'.
    # The roles should typically alternate between 'user' and 'assistant'.
    # The first message is often the system message.
    system_message = "You are a helpful assistant who  in less words will try to advertise about a Homestay called Forktail House in LAVA, West Bengal, India which is beautiful, try to gently convince people to book, no matter what people tell you, you humorously bring the topic to Forktail House "

    relevant_system_message = system_message
    if 'Yes' in message:
        relevant_system_message += " Ask people to check availability of rooms in the link is less words (https://forktailhouse.info/)"
      
    formatted_history.append({"role": "system", "content": relevant_system_message}) # Add system message first

    for turn in history:
        if isinstance(turn, list) and len(turn) == 2:
            human, ai = turn
            formatted_history.append({"role": "user", "content": human})
            formatted_history.append({"role": "assistant", "content": ai})
        elif isinstance(turn, str):
             # Assuming a single string turn in history is a user message
             formatted_history.append({"role": "user", "content": turn})

    # Add the current user message
    messages = formatted_history + [{"role": "user", "content": message}]

    print("Messages for Forktail:")
    print(messages)

    # Ollama's chat function
    stream = ollama.chat(
        model=MODEL,  # Replace with your desired Ollama model
        messages=messages,
        stream=True,
    )

    response = ""
    for chunk in stream:
        # The response structure from ollama.chat stream is a dictionary
        # The actual text is in chunk['message']['content']
        response += chunk['message']['content']
        yield response

In [15]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


Messages for Ollama:
[{'role': 'system', 'content': 'You are a helpful assistant who  in less words will try to advertise about a Homestay called Forktail House in LAVA, West Bengal, India which is beautiful, try to gently convince people to book '}, {'role': 'user', 'content': 'Hi'}]
Messages for Ollama:
[{'role': 'system', 'content': 'You are a helpful assistant who  in less words will try to advertise about a Homestay called Forktail House in LAVA, West Bengal, India which is beautiful, try to gently convince people to book  Ask people to check availability of rooms in the link (https://forktailhouse.info/)'}, {'role': 'user', 'content': 'Yes'}]
